In [2]:
:dep polars = "0.36.2"
:dep linregress = "0.5.3"

In [3]:
use std::path::Path;
use polars::prelude::*;
use linregress::{FormulaRegressionBuilder, RegressionDataBuilder};

In [4]:
let df = DataFrame::default();
df

shape: (0, 0)
┌┐
╞╡
└┘

In [83]:
let s1 = Series::new("date", &["2024-01-01", "2024-01-02", "2024-01-03"]);
let s2 = Series::new("identifier", &["Ali", "Ali", "Ali"]);
let s3 = Series::new("residual", &[Some(1.84), Some(1.78), None::<f64>]);
// let s3 = series![1.84, 1.78, None::<f64>];
// let s4 = Series::new("sum_res", &[67, 78, 43]);
let s4 = Series::new("sum_res", &[Some(67), Some(78), None::<i32>]);
let s5 = Series::new("sum_res_lag", &[23, 27, 47]);
let s6 = Series::new("a", &[1.84, 1.78, 1.23]);
let s7 = Series::new("b1", &[23, 27, 33]);
let s8 = Series::new("rss", &[1.84, 1.78, 1.67]);

let mut df: DataFrame = DataFrame::new(vec![s1, s2, s3, s4, s5, s6, s7, s8]).unwrap();
df

shape: (3, 8)
┌────────────┬────────────┬──────────┬─────────┬─────────────┬──────┬─────┬──────┐
│ date       ┆ identifier ┆ residual ┆ sum_res ┆ sum_res_lag ┆ a    ┆ b1  ┆ rss  │
│ ---        ┆ ---        ┆ ---      ┆ ---     ┆ ---         ┆ ---  ┆ --- ┆ ---  │
│ str        ┆ str        ┆ f64      ┆ i32     ┆ i32         ┆ f64  ┆ i32 ┆ f64  │
╞════════════╪════════════╪══════════╪═════════╪═════════════╪══════╪═════╪══════╡
│ 2024-01-01 ┆ Ali        ┆ 1.84     ┆ 67      ┆ 23          ┆ 1.84 ┆ 23  ┆ 1.84 │
│ 2024-01-02 ┆ Ali        ┆ 1.78     ┆ 78      ┆ 27          ┆ 1.78 ┆ 27  ┆ 1.78 │
│ 2024-01-03 ┆ Ali        ┆ null     ┆ null    ┆ 47          ┆ 1.23 ┆ 33  ┆ 1.67 │
└────────────┴────────────┴──────────┴─────────┴─────────────┴──────┴─────┴──────┘

In [77]:
let mut total_null_count = 0;
let df_selected = df.select(&["sum_res", "sum_res_lag"]).unwrap();

for col in df_selected.get_columns(){
    let col_null_count = col.is_null().sum();
    let int_val: i32 = col_null_count.unwrap_or(0) as i32;
    total_null_count += int_val;
    println!("{}", total_null_count);
}
    
    

1
1


()

In [78]:
fn null_count(df: DataFrame, cols: Vec<String>) -> Result<i32, PolarsError>{
    let mut total_null_count = 0;
    let df_selected = df.select(&cols).unwrap();
    
    for col in df_selected.get_columns(){
        let col_null_count = col.is_null().sum();
        let int_val: i32 = col_null_count.unwrap_or(0) as i32;
        total_null_count += int_val;
    }
    Ok(total_null_count)
}

In [79]:
let columns = vec!["sum_res".to_string(), "sum_res_lag".to_string()];
let mut count = 0;
match null_count(df, columns) {
    Ok(count) => println!("Total null count: {}", count),
    Err(err) => eprintln!("Error: {}", err),
}
count

Total null count: 1


0

In [80]:
count

0

In [85]:
let columns = vec!["sum_res".to_string(), "sum_res_lag".to_string()];
let result = null_count(df.clone(), columns.clone());

In [86]:
// Declare the count variable outside the match
let mut count = 0;

// Handle the result
match result {
    Ok(c) => {
        // Assign the value to the count variable
        count = c;
        println!("Total null count: {}", count);
    }
    Err(err) => eprintln!("Error: {}", err),
}

// Now you can use 'count' variable here
println!("Count outside match: {}", count);

Total null count: 1
Count outside match: 1


In [87]:
// let X = df.select(&["sum_res"]).expect("Column 'sum_res' not found");
let ones = Series::new("ones", &vec![1; df.height()]);
ones

shape: (3,)
Series: 'ones' [i32]
[
	1
	1
	1
]

In [88]:
df.replace("a", Series::new("a", &vec![9; df.height()]));
df

shape: (3, 8)
┌────────────┬────────────┬──────────┬─────────┬─────────────┬─────┬─────┬──────┐
│ date       ┆ identifier ┆ residual ┆ sum_res ┆ sum_res_lag ┆ a   ┆ b1  ┆ rss  │
│ ---        ┆ ---        ┆ ---      ┆ ---     ┆ ---         ┆ --- ┆ --- ┆ ---  │
│ str        ┆ str        ┆ f64      ┆ i32     ┆ i32         ┆ i32 ┆ i32 ┆ f64  │
╞════════════╪════════════╪══════════╪═════════╪═════════════╪═════╪═════╪══════╡
│ 2024-01-01 ┆ Ali        ┆ 1.84     ┆ 67      ┆ 23          ┆ 9   ┆ 23  ┆ 1.84 │
│ 2024-01-02 ┆ Ali        ┆ 1.78     ┆ 78      ┆ 27          ┆ 9   ┆ 27  ┆ 1.78 │
│ 2024-01-03 ┆ Ali        ┆ null     ┆ null    ┆ 47          ┆ 9   ┆ 33  ┆ 1.67 │
└────────────┴────────────┴──────────┴─────────┴─────────────┴─────┴─────┴──────┘

In [ ]:
fn rolling_reg(df: DataFrame, n: i32) -> Result<DataFrame, PolarsError>{
    """
    df: df must be mutable dataframe, index is RangeIndex, columns are ['date', 'identifier', 'residual', 'sum_res', 'sum_res_lag', 'a', 'b1', 'rss']
    """
    let columns = vec!["sum_res".to_string(), "sum_res_lag".to_string()];
    for i in n..df.height(){
        let mut temp: DataFrame = df.slice(i-n, i);
        let result = null_count(temp.clone(), columns.clone());
        let mut count = 0;
        // Handle the result
        match result {
            Ok(c) => {
                // Assign the value to the count variable
                count = c;
            }
            Err(err) => eprintln!("Error: {}", err),
        }
        if count == 0{
            // Extract Series for dependent and response variables
            let y = temp.column("sum_res_lag").expect("Column 'sum_res_lag' not found");
            let X = temp.select(&["sum_res"]).expect("Column 'sum_res' not found");

            // Fit linear regression model with constant
            let ones = Series::new("ones", &[1; temp.height() as usize]);
            let X_with_ones = X.hstack(&[ones]);

            // Calculate coefficients manually
            let X_transposed = X_with_ones.transpose().expect("Error transposing X");
            let XTX_inv = X_transposed.dot(&X_with_ones).expect("Error calculating XTX_inv").inv().expect("Error inverting XTX_inv");
            let coefficients = XTX_inv.dot(&X_transposed).expect("Error calculating coefficients").dot(y).expect("Error calculating y");

            // Create a new DataFrame with coefficients and intercept
            let coefficients_series = coefficients.into_series();
            temp.replace("a", Series::new("a", &vec![coefficients[1]; df.height()]));
            temp.replace("b1", Series::new("b1", &vec![coefficients[2]; df.height()]));
            temp.replace("rss", Series::new("rss", &vec![coefficients[0]; df.height()]));
        }
        
    }
}